# Example of accessing live point detectors and ROI analysis
WARNING: Execute these python scripts slowly. Otherwise the execution hangs.

# Import required libraries

In [1]:
import os, io, time, sys, socket
import numpy as np
import pandas as pd

# SACLA database library and online analysis library
sys.path.append('/home/software/SACLA_tool/local/python3.5/lib/python3.5/site-packages')
import dbpy, olpy

# Import custom online library
import onlineAccess


# Plot options
# Plots in an external window and not within Jupyter.
import matplotlib
matplotlib.rcParams['backend']='Qt5Agg'
matplotlib.use('Qt5Agg')
%matplotlib qt5
import matplotlib.pyplot as plt
import matplotlib.widgets as widgets


MPCCD threshold is set to 1369.863014


# Setup data grabber thread

In [2]:
# Specify number of events to grab data against at a time
ngrab = 120

# Specify the maximum number of events/tags to save in the thread deques
maxTags2Save = 100000

# Specify the beamline you are working on
bl = 3

# Specify a detector to use for reference to get newest event tags
refDet='xfel_bl_3_st_5_direct_bm_1_pd/charge'

# Generate the dataHandler thread for getting data
dh = onlineAccess.dataHandler(  bl=bl, 
                              refDet=refDet, 
                              ngrab=ngrab, 
                              maxTags2Save = maxTags2Save )

# Lookup available array detectors

In [10]:
onlineAccess.getCurrentDetectorList(3)

('MPCCD-1N0-M06-004',
 'MPCCD-2N0-1-006',
 'MPCCD-2N0-1-006-1',
 'MPCCD-2N0-1-006-2')

# Lookup available point detectors 

In [ ]:
onlineAccess.getEquipmentList()

# Specify the point detectors and ROIs to grab

In [3]:
# point detectors are input as a list. 
# These names can be found using the function above or in the web interface for looking up data
# on the opcon computers.

pointDetectors = ['xfel_bl_3_st_5_direct_bm_1_pd/charge',
                 'xfel_bl_3_tc_bm_1_pd/charge',
                 'xfel_bl_3_tc_bm_2_pd/charge', 
                 'xfel_bl_3_tc_spec_1/energy',
                 'xfel_mon_bpm_bl3_0_3_beamstatus/summary',
                 'xfel_bl_3_shutter_1_open_valid/status',
                 'xfel_bl_3_st_5_motor_user_1/position',
                 'xfel_bl_3_st_5_motor_user_2/position',
                 'xfel_bl_3_st_5_motor_user_3/position',
                 'xfel_bl_3_st_5_motor_user_4/position',
                 'xfel_bl_3_st_5_motor_user_5/position',
                 'xfel_bl_3_st_5_motor_user_15/position',
                 'xfel_bl_3_st_5_motor_facility_14/position',
                 'xfel_bl_3_st_5_motor_user_18/position',
                 'xfel_bl_3_st_5_motor_facility_10/position',
                 'xfel_bl_3_st_5_pd_user_5_fitting_peak/voltage']

dh.setPointDetector( pointDetectors )

# point detectors are input as a list. 
# These names can be found using the function above or in the web interface for looking up data
# on the opcon computers.
rois = { 'fullSingle': {'Detector':'MPCCD-1N0-M06-004',
                 'X1':0,
                 'X2':511,
                 'Y1':0,
                 'Y2':1023} } 

# COMMENT ME OUT IF THE DETECTOR IS NOT SETUP. OTHERWISE CODE WILL CRASH.
# If you'd like to debug code before experiment, there exist an anapc simulation environment.
dh.setROIs(rois)

### Begin execution of thread

In [4]:
dh.start()

### Request termination of thread (leave commented, except when you want to kill thread)

In [5]:
# dh.requestStop()

### Print status of thread

In [6]:
print(dh.status)
print("running for %f s"%dh.elapse)

Waiting for initialization, point detectors initialized, rois initialized, running
running for 1.565703 s


### Print names of detectors / rois being saved

In [7]:
print("Keys being grabbed:")
print(list(dh.dequeDicts.keys()))

Keys being grabbed:
['xfel_bl_3_st_5_direct_bm_1_pd/charge', 'xfel_bl_3_st_5_pd_user_5_fitting_peak/voltage', 'xfel_mon_bpm_bl3_0_3_beamstatus/summary', 'xfel_bl_3_st_5_motor_user_5/position', 'dummy', 'xfel_bl_3_st_5_motor_user_4/position', 'xfel_bl_3_tc_spec_1/energy', 'xfel_bl_3_st_5_motor_user_3/position', 'xfel_bl_3_st_5_motor_user_15/position', 'tags', 'xfel_bl_3_tc_bm_1_pd/charge', 'xfel_bl_3_shutter_1_open_valid/status', 'xfel_bl_3_st_5_motor_user_1/position', 'xfel_bl_3_tc_bm_2_pd/charge', 'xfel_bl_3_st_5_motor_user_2/position', 'fullSingle', 'xfel_bl_3_st_5_motor_user_18/position', 'xfel_bl_3_st_5_motor_facility_14/position', 'xfel_bl_3_st_5_motor_facility_10/position']


# Setup plotting

In [9]:
plotEveryNtags = 2000

def isData( anArray ):
    return (np.abs(anArray)>0) & (~np.isnan(anArray))
'''
The function below specifies the analysis and plot setup.
'''
def updatePlots( ax , dh , plotLast = 2000, photonEnergykeV = 1, solidAngle = 1  ):
    # Tags
    tags = dh['tags'][-plotLast:]

    # Beam status
    beamCharge = dh['xfel_bl_3_st_5_direct_bm_1_pd/charge'][-plotLast:]

    # Grab only nonzero indexes
    beamOn = isData(beamStatus)

    # Beam charge v MPCCD
    idxs = isData(beamCharge) & beamOn 
    ax.scatter( tags[idxs] ,(beamCharge[idxs]*1.e9), s=3, c='b' ) 
    ax.set_xlabel('tags')
    ax.set_ylabel('beamCharge')
    
    

# Run plotting

In [ ]:
# Setup the figure and axes for plotting
fig,ax=plt.subplots()
plt.axis('off')
fig.show()
figure_open=True

# Tells loop to exit if the figure is closed
def close_event_handler(evt):
    global figure_open
    figure_open=False
    print("closed figure")
fig.canvas.mpl_connect('close_event',close_event_handler)   

# Figure updated every ngrab/30. seconds
while figure_open:
    # Pause data grabber while plotting
    dh.pause()
    while dh.isPaused is False:
        continue
    
    #add plots here
    ax.clear()
    updatePlots( ax , dh , plotLast = plotEveryNtags, photonEnergykeV = photonEnergykeV, solidAngle = solidAngle  )

    # Restart data grabber
    dh.restart()  
    t0 = time.time()
    
    fig.canvas.draw()
    plt.pause(ngrab/30.)
    
    
print("done plotting")

/home/mrware/anaconda3/lib/python3.5/site-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)
